## Format and Label VEP annotated WGS results for CAD12 and CAD63
**last update : july 9 2024 3pm**

### code version info 
1.  clinvar col with labels is not star reviewed, v1overestimates  number of pathogenic variants because it adds label if the phrase occurs. for high confidendence pathogenic variants, check the clin.sig column to confirm absence of 'benign' or 'uncertain significance' phrases (DONE)
2. added Database column with python instead of  R to keep all label functions together (DONE)
3. note: keyID38 is only for SNVs
4. note: var.in.both.lines has T/F values
5. note: gene.in.both.lines has T/F values
6. note: MAF is based on MAX.AF column

In [1]:
from all_funx import *      # conda env  py39 (Python 3.9.-1)

In [2]:

# import dfs from 1_clean_excel_summary.py

path  = "/Users/mariapalafox/Desktop/BOLDcaution/WGS/5-annotation/3-interpret/"


# import cad 12 wgs results for snp indel

indel12 = pd.read_csv('1_clean_vep_output-indel-cad12-v2.csv', low_memory=False)
snp12 = pd.read_csv('1_clean_vep_output-snp-cad12-v2.csv', low_memory=False)

print(indel12.shape)    # (546104, 80)
print(snp12.shape)      # (2950097, 85)


# import cad 63 wgs results for snp indel

indel63 = pd.read_csv('1_clean_vep_output-indel-cad63-v2.csv', low_memory=False)
snp63 = pd.read_csv('1_clean_vep_output-snp-cad63-v2.csv', low_memory=False)

print(indel63.shape)        # (562353, 80)
print(snp63.shape)          # (3138661, 85)


(546104, 80)
(2950097, 85)
(562353, 80)
(3138661, 85)


## Result. CAD63 has 16,249 more INDELs and 188,564 more SNVs than CAD12 :

* cad12 :
    * INDEL = 546,104
    * SNV = 2,950,097

* cad63 :
    * INDEL = 562,353
    * SNV = 3,138,661

* difference INDEL : 562353-546104 = 16249

* difference SNV : 3138661-2950097 = 188564
---
---

In [3]:
# make cad12 and cad63 complete dfs

cad12 = pd.concat([indel12, snp12], axis=0)
cad63 = pd.concat([indel63, snp63], axis=0)

print(cad12.shape)
print(cad63.shape)

(3496201, 85)
(3701014, 85)



---

# [1]  label  Exclusive VARIANT and Exclusive SYMBOL with variant btw CAD12 vs CAD63

---

In [4]:
# create list of unique 'Uploaded_variation'

cad12_variations = cad12['Uploaded_variation'].unique()
cad63_variations = cad63['Uploaded_variation'].unique()

# create  list for  exclusively in cad12

cad12_exclusive_variations = list(set(cad12_variations) - set(cad63_variations))


# create  list for  exclusively in cad63

cad63_exclusive_variations = list(set(cad63_variations) - set(cad12_variations))

# create list for in both lines

common_variations = list(set(cad12_variations) & set(cad63_variations))


print( len(cad12_exclusive_variations), len(cad63_exclusive_variations), len(common_variations)) 

1525780 1730593 1970421


## Result.Exclusive and Shared VARIANT :

###  CAD12 minus CAD63 =    1,525,780
###  CAD63 minus CAD12 =    1,730,593
## Shared =                 1,970,421
---
---

In [5]:
#  dfs for CAD12 and CAD63

# # TODO drop redundant column labeling function calls

cad12['var_in_both_lines'] = cad12['Uploaded_variation'].isin(common_variations)
cad12['var_exclusive_btw_lines'] = cad12['Uploaded_variation'].isin(cad12_exclusive_variations)

cad63['var_in_both_lines'] = cad63['Uploaded_variation'].isin(common_variations)
cad63['var_exclusive_btw_lines'] = cad63['Uploaded_variation'].isin(cad63_exclusive_variations)

In [6]:
print('CAD12')
checkColumnsValues(cad12, ['var_in_both_lines', 'var_exclusive_btw_lines'])
print()
print()
print()
print('CAD63')
checkColumnsValues(cad63, ['var_in_both_lines', 'var_exclusive_btw_lines'])

CAD12
['var_in_both_lines', 'var_exclusive_btw_lines']

var_in_both_lines        False    1525780
                         True     1970421
var_exclusive_btw_lines  False    1970421
                         True     1525780
dtype: int64




CAD63
['var_in_both_lines', 'var_exclusive_btw_lines']

var_in_both_lines        False    1730593
                         True     1970421
var_exclusive_btw_lines  False    1970421
                         True     1730593
dtype: int64



**qc note : same counts  as above**


In [7]:
# create list of unique 'SYMBOL' in both dataframes
cad12_genes = cad12['SYMBOL'].unique()
cad63_genes = cad63['SYMBOL'].unique()

# creat list for SYMBOL  both cad12 and cad63
common_genes = list(set(cad12_genes) & set(cad63_genes))

# create list for SYMBOL   exclusively in cad12
cad12_exclusive = list(set(cad12_genes) - set(cad63_genes))
cad12['gene_in_both_lines'] = cad12['SYMBOL'].isin(common_genes)
cad12['gene_exclusive_btw_lines'] = cad12['SYMBOL'].isin(cad12_exclusive)

# create list for SYMBOL   exclusively in cad63
cad63_exclusive = list(set(cad63_genes) - set(cad12_genes))
cad63['gene_in_both_lines'] = cad63['SYMBOL'].isin(common_genes)
cad63['gene_exclusive_btw_lines'] = cad63['SYMBOL'].isin(cad63_exclusive)


print( len(cad12_exclusive), len(cad63_exclusive), len(common_genes)) 
#809 743 28137

809 743 28137


## Result.Exclusive and Shared SYMBOL : 

###  CAD12 Exclusive  =     809
###  CAD63 mExclusive   =   743
### Shared =                 28,137
---
---

In [8]:

print(cad12.shape)      # (3496201, 89)
print(cad63.shape)      #(3701014, 89)

(3496201, 89)
(3701014, 89)


## Result.Exclusive and Shared SYMBOL *(REDUNDANT - counts are unique to variant level NOT gene level and below is variant level counts)*

```python
checkColumnsValues(cad12, ['gene_in_both_lines', 'gene_exclusive_btw_lines'])
print()
print()
print()
print()
checkColumnsValues(cad63, ['gene_in_both_lines', 'gene_exclusive_btw_lines'])
```

```
---
### CAD12 Shared =                3,491,756
###  CAD12 Exclusive  =           4,445
### CAD12 TOTAL shape =             3,496,201
---
### CAD63 Shared =                3,697,611
###  CAD63 Exclusive   =          3,403
### CAD63 TOTAL shape =             3,701,014
```

--- 
---

In [9]:
# fix column names by replacing _ with .

cad12.columns = cad12.columns.str.replace('_', '.')
cad63.columns = cad63.columns.str.replace('_', '.')


---

# [2] label  gnomad MAF thresholds

---

In [10]:
# add MAF column 3 groups :  if MAX.AF is < 0.01 label Rare, if less than 0.1 label less common, if greater than or equal  to 0.1 label common, else label empty

cad12['MAF'] = np.select([cad12['MAX.AF'] < 0.01, 
                          (cad12['MAX.AF'] >= 0.01) & (cad12['MAX.AF'] < 0.1),
                            cad12['MAX.AF'] >= 0.1], 
                            ['Rare (<1%)', 'Less common (<10%)', 'Common (>=10%)'], 
                            default = np.nan)

cad63['MAF'] = np.select([cad63['MAX.AF'] < 0.01, 
                          (cad63['MAX.AF'] >= 0.01) & (cad63['MAX.AF'] < 0.1), 
                          cad63['MAX.AF'] >= 0.1], 
                          ['Rare (<1%)', 'Less common (<10%)', 'Common (>=10%)'], 
                          default = np.nan)

In [11]:
# add MAF2 column 2 groups:  if MAX.AF is < 0.05 label Rare,  if greater than or equal  to 0.05 label  Common, else label empty

cad12['MAF2'] = np.select([cad12['MAX.AF'] < 0.05, 
                           cad12['MAX.AF'] >= 0.05], 
                           ['Rare (<5%)', 'Common (>=5%)'],
                            default=np.nan)

cad63['MAF2'] = np.select([cad63['MAX.AF'] < 0.05, 
                           cad63['MAX.AF'] >= 0.05], 
                           ['Rare (<5%)', 'Common (>=5%)'], 
                           default=np.nan)

## Result. gnomad MAF label counts :

In [12]:
print("CAD12")
checkColumnsValues(cad12, ['MAF', 'MAF2'])
print()
print()
print()
print()
print("CAD63")
checkColumnsValues(cad63, ['MAF', 'MAF2'])


CAD12
['MAF', 'MAF2']

MAF   Common (>=10%)        3253032.0
      Less common (<10%)     152860.0
      Rare (<1%)              54530.0
      nan                     35779.0
MAF2  Common (>=5%)         3326323.0
      Rare (<5%)             134099.0
      nan                     35779.0
dtype: float64





CAD63
['MAF', 'MAF2']

MAF   Common (>=10%)        3462469.0
      Less common (<10%)     157624.0
      Rare (<1%)              54790.0
      nan                     26131.0
MAF2  Common (>=5%)         3542705.0
      Rare (<5%)             132178.0
      nan                     26131.0
dtype: float64



---
#  [3] label PATHO, BENIGN, VUS if any word found in CLIN.SIG & label ClinVar final based on hierarchy
---

In [13]:
# call add  ClinVar label functions

def label_clin_sig(df):
    # function creates 3 new cols - PATHO VUS and BENIGN by checking CLIN.SIG  for i.e.  'pathogenic' or 'likely pathogenic'   new column  'PATHO' label boolean
    df['PATHO'] = df['CLIN.SIG'].str.contains('pathogenic|likely pathogenic', case=False)
    df['VUS'] = df['CLIN.SIG'].str.contains('uncertain_significance', case=False)
    df['BENIGN'] = df['CLIN.SIG'].str.contains('benign|likely_benign', case=False)
    return df

def label_clin_sig2(df):
    # function creates one column ClinVar label final based on  heirarchy;
         # ## v1 label_clin_sig2() commented out:        over-estimated number of pathogenic because if-else statement ordered by PATHO check first then VUS then BENIGN
        # ## v2 label_clin_sig2() current version:      high Confidence number of pathogenic because if-else statement ordered by VUS first then BENIGN then PATHO
    newcol = []
    for row in df.itertuples():
        if row.VUS:
            newcol.append('VUS')
        elif row.BENIGN:
            newcol.append('BENIGN')
        elif row.PATHO:
            newcol.append('PATHO')
        else:
            newcol.append(np.nan) # append nan value instead of '' emptry string

    df['ClinVar'] = newcol
    return df




# done TODO  new column should  contain 'VUS' and 'BENIGN' labels if the row contain 'benign' for 'BENIGN' and 'uncertain significance' for 'VUS'.

cad12 = label_clin_sig(cad12)
cad12 = label_clin_sig2(cad12)
cad63 = label_clin_sig(cad63)
cad63 = label_clin_sig2(cad63)

## Result. CAD12 ClinVar Label (if patho checked FIRST) :

In [14]:
checkColumnValue(cad12, 'ClinVar')      # QC v1

,Count,count
0,NaN,3464042
1,BENIGN,31835
2,VUS,309
3,PATHO,15


In [15]:

crosstabit(cad12, 'IND', 'ClinVar', True)  # QC v1

ClinVar,BENIGN,NA,PATHO,VUS,Total
IND,,,,,
P732912,31835,3464042,15,309,3496201
Total,31835,3464042,15,309,3496201


## Result. CAD12 ClinVar Label (if patho checked LAST) : 

In [16]:
checkColumnValue(cad12, 'ClinVar')

,Count,count
0,NaN,3464042
1,BENIGN,31835
2,VUS,309
3,PATHO,15


In [17]:
crosstabit(cad12, 'IND', 'ClinVar', True) 

ClinVar,BENIGN,NA,PATHO,VUS,Total
IND,,,,,
P732912,31835,3464042,15,309,3496201
Total,31835,3464042,15,309,3496201


In [18]:
patho12 = cad12[cad12['ClinVar'] == 'PATHO']

In [19]:
crosstabit(patho12, 'IND', 'PATHO', True) # qc 

PATHO,True,Total
IND,,
P732912,15,15
Total,15,15


In [20]:
crosstabit(patho12, 'IND', 'BENIGN', True) # qc 

BENIGN,False,Total
IND,,
P732912,15,15
Total,15,15


In [21]:
crosstabit(patho12, 'IND', 'VUS', True) # qc 

VUS,False,Total
IND,,
P732912,15,15
Total,15,15


---
---

## Result. CAD63 ClinVar Label (if patho labeled FIRST)  :

In [22]:
checkColumnValue(cad63, 'ClinVar')

,Count,count
0,NaN,3667535
1,BENIGN,33116
2,VUS,346
3,PATHO,17


In [23]:
crosstabit(cad63, 'IND', 'ClinVar', True)

ClinVar,BENIGN,NA,PATHO,VUS,Total
IND,,,,,
P931263,33116,3667535,17,346,3701014
Total,33116,3667535,17,346,3701014


## Result. CAD63 ClinVar Label (if patho labeled LAST)  : 

In [24]:
checkColumnValue(cad63, 'ClinVar')

,Count,count
0,NaN,3667535
1,BENIGN,33116
2,VUS,346
3,PATHO,17


In [25]:
crosstabit(cad63, 'IND', 'ClinVar', True)

ClinVar,BENIGN,NA,PATHO,VUS,Total
IND,,,,,
P931263,33116,3667535,17,346,3701014
Total,33116,3667535,17,346,3701014


In [26]:
patho63 = cad63[cad63['ClinVar'] == 'PATHO']

In [27]:
crosstabit(patho63, 'IND', 'PATHO', True)   # qc 

PATHO,True,Total
IND,,
P931263,17,17
Total,17,17


In [28]:
crosstabit(patho63, 'IND', 'BENIGN', True)  # qc 

BENIGN,False,Total
IND,,
P931263,17,17
Total,17,17


In [29]:
crosstabit(patho63, 'IND', 'VUS', True) # qc 

VUS,False,Total
IND,,
P931263,17,17
Total,17,17


---
---

In [30]:
keepme = ['IND','Uploaded.variation',    # 'SYMBOL.SOURCE',
             'SYMBOL' ,  'gene.in.both.lines',   'ZYG',  

          'VARIANT.CLASS', 'Consequence',   'Amino.acids', #'BIOTYPE',

           'var.in.both.lines',
          
             'ClinVar',   'MAF', 'MAF2', 
           
             'CLIN.SIG',  #'PATHO', 'VUS', 'BENIGN',
        
           'MAX.AF', 'MAX.AF.POPS', 
   
           #'SOMATIC', 'PHENO', 'PUBMED',

            'SIFT', 'PolyPhen',  
              'SIFTprediction', #'SIFTscore',
                'PolyPhenprediction', #'PolyPhenscore', 

            'TRANSCRIPTION.FACTORS',  'miRNA',  'DOMAINS', 'MOTIF.NAME', 'MOTIF.POS', 'MOTIF.SCORE.CHANGE', 'HIGH.INF.POS'  ]

## Result. total ClinVar per CAD12 and CAD63 : 

In [31]:
checkColumnValue(cad12, 'ClinVar')

,Count,count
0,NaN,3464042
1,BENIGN,31835
2,VUS,309
3,PATHO,15


In [32]:
patho12 = cad12[cad12['ClinVar'] == 'PATHO']
print(patho12.shape)
patho12[keepme].sort_values(['SYMBOL'])

(15, 95)


,IND,Uploaded.variation,SYMBOL,gene.in.both.lines,ZYG,VARIANT.CLASS,Consequence,Amino.acids,var.in.both.lines,ClinVar,MAF,MAF2,CLIN.SIG,MAX.AF,MAX.AF.POPS,SIFT,PolyPhen,SIFTprediction,PolyPhenprediction,TRANSCRIPTION.FACTORS,miRNA,DOMAINS,MOTIF.NAME,MOTIF.POS,MOTIF.SCORE.CHANGE,HIGH.INF.POS
1636481,P732912,chr10_6080046_T/A,-,True,HOM,SNV,intergenic_variant,-,False,PATHO,Common (>=10%),Common (>=5%),pathogenic,0.394600,gnomADg_ASJ,-,-,-,-,-,-,-,-,-,-,-
81986,P732912,chr1_75761116_T/G,ACADM,True,HET,SNV,"splice_region_variant,splice_polypyrimidine_tr...",-,False,PATHO,Rare (<1%),Rare (<5%),pathogenic,0.000654,gnomADe_SAS,-,-,-,-,-,-,-,-,-,-,-
2499653,P732912,chr17_63491327_G/A,ACE,True,HET,SNV,missense_variant,R/Q,False,PATHO,Less common (<10%),Rare (<5%),conflicting_interpretations_of_pathogenicity,0.012300,SAS,deleterious(0),benign(0.153),deleterious,benign,-,-,"PDB-ENSP_mappings:1o86.A,PDB-ENSP_mappings:1o8...",-,-,-,-
139169,P732912,chr1_161223893_G/A,APOA2,True,HET,SNV,upstream_gene_variant,-,False,PATHO,Common (>=10%),Common (>=5%),pathogenic,0.908800,gnomADg_EAS,-,-,-,-,-,-,-,-,-,-,-
269741,P732912,chr9_27573524_GCCCCG/GCCCCGGCCCCGGCCCCGGCCCCGG...,C9orf72,True,HET,sequence_alteration,upstream_gene_variant,-,False,PATHO,Common (>=10%),Common (>=5%),pathogenic,0.133000,gnomADg_EAS,-,-,NaN,NaN,-,-,-,-,-,-,-
2760537,P732912,chr20_62837227_G/A,COL9A3,True,HET,SNV,missense_variant,R/H,False,PATHO,Rare (<1%),Rare (<5%),conflicting_interpretations_of_pathogenicity,0.001000,EAS,deleterious(0.02),probably_damaging(0.994),deleterious,probably_damaging,-,-,"AFDB-ENSP_mappings:AF-Q14050-F1.A,Pfam:PF01391...",-,-,-,-
59969,P732912,chr1_53906216_G/A,DIO1,True,HET,SNV,missense_variant,M/I,False,PATHO,Less common (<10%),Rare (<5%),pathogenic,0.024500,SAS,deleterious(0.02),probably_damaging(0.978),deleterious,probably_damaging,-,-,"Gene3D:3.40.30.10,Pfam:PF00837,PIRSF:PIRSF0013...",-,-,-,-
1333456,P732912,chr7_150742750_T/C,GIMAP5,True,HET,SNV,missense_variant,L/P,False,PATHO,Rare (<1%),Rare (<5%),"likely_pathogenic,pathogenic",0.003175,gnomADg_NFE,deleterious(0),probably_damaging(0.992),deleterious,probably_damaging,-,-,"Gene3D:3.40.50.300,PDB-ENSP_mappings:6z3e.A,AF...",-,-,-,-
1311251,P732912,chr7_128938247_T/G,IRF5,True,HET,SNV,intron_variant,-,True,PATHO,Common (>=10%),Common (>=5%),"pathogenic,risk_factor",0.749000,EAS,-,-,-,-,-,-,-,-,-,-,-
2626072,P732912,chr19_15187315_G/A,NOTCH3,True,HET,SNV,missense_variant,R/C,False,PATHO,Rare (<1%),Rare (<5%),"pathogenic,pathogenic/likely_pathogenic,likely...",0.004140,gnomADe_EAS,tolerated(0.05),possibly_damaging(0.457),tolerated,possibly_damaging,-,-,"Gene3D:2.10.25.10,AFDB-ENSP_mappings:AF-Q9UM47...",-,-,-,-


In [33]:
# patho12 = cad12[cad12['ClinVar'] == 'PATHO']
# print(patho12.shape)
# patho12[keepme].sort_values(['SYMBOL'])

benign12 = cad12[cad12['ClinVar'] == 'BENIGN']
print(benign12.shape)
benign12[keepme].sort_values(['SYMBOL'])

(31835, 95)


,IND,Uploaded.variation,SYMBOL,gene.in.both.lines,ZYG,VARIANT.CLASS,Consequence,Amino.acids,var.in.both.lines,ClinVar,MAF,MAF2,CLIN.SIG,MAX.AF,MAX.AF.POPS,SIFT,PolyPhen,SIFTprediction,PolyPhenprediction,TRANSCRIPTION.FACTORS,miRNA,DOMAINS,MOTIF.NAME,MOTIF.POS,MOTIF.SCORE.CHANGE,HIGH.INF.POS
2430310,P732912,chr16_86184895_G/A,-,True,HOM,SNV,intergenic_variant,-,False,BENIGN,Common (>=10%),Common (>=5%),benign,0.6793,AFR,-,-,-,-,-,-,-,-,-,-,-
1258708,P732912,chr7_67771556_T/C,-,True,HOM,SNV,upstream_gene_variant,-,True,BENIGN,Common (>=10%),Common (>=5%),benign,0.7357,gnomADg_SAS,-,-,-,-,-,-,-,-,-,-,-
1258714,P732912,chr7_67772928_A/C,-,True,HOM,SNV,upstream_gene_variant,-,True,BENIGN,Common (>=10%),Common (>=5%),benign,0.7133,gnomADg_SAS,-,-,-,-,-,-,-,-,-,-,-
2378574,P732912,chr16_30067171_G/C,-,True,HOM,SNV,intron_variant,-,False,BENIGN,Common (>=10%),Common (>=5%),benign,0.6180,AFR,-,-,-,-,-,-,-,-,-,-,-
691521,P732912,chr4_26450075_G/A,-,True,HET,SNV,intergenic_variant,-,True,BENIGN,Common (>=10%),Common (>=5%),benign,0.6931,AMR,-,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160281,P732912,chr5_61494146_GG/-,ZSWIM6,True,HET,deletion,intron_variant,-,True,BENIGN,Common (>=10%),Common (>=5%),benign,0.4555,gnomADg_AMI,-,-,NaN,NaN,-,-,-,-,-,-,-
904452,P732912,chr5_61539194_T/C,ZSWIM6,True,HET,SNV,intron_variant,-,True,BENIGN,Common (>=10%),Common (>=5%),benign,0.4632,SAS,-,-,-,-,-,-,-,-,-,-,-
160287,P732912,chr5_61543228_-/T,ZSWIM6,True,HET,insertion,intron_variant,-,True,BENIGN,Common (>=10%),Common (>=5%),benign,0.4484,gnomADg_AMI,-,-,NaN,NaN,-,-,-,-,-,-,-
904449,P732912,chr5_61535754_T/G,ZSWIM6,True,HET,SNV,intron_variant,-,True,BENIGN,Common (>=10%),Common (>=5%),benign,0.4594,gnomADg_AMI,-,-,-,-,-,-,-,-,-,-,-


In [34]:
# patho12 = cad12[cad12['ClinVar'] == 'PATHO']
# print(patho12.shape)
# patho12[keepme].sort_values(['SYMBOL'])

# benign12 = cad12[cad12['ClinVar'] == 'BENIGN']
# print(benign12.shape)
# benign12[keepme].sort_values(['SYMBOL'])

vus12 = cad12[cad12['ClinVar'] == 'VUS']
print(vus12.shape)
vus12[keepme].sort_values(['SYMBOL'])

(309, 95)


,IND,Uploaded.variation,SYMBOL,gene.in.both.lines,ZYG,VARIANT.CLASS,Consequence,Amino.acids,var.in.both.lines,ClinVar,MAF,MAF2,CLIN.SIG,MAX.AF,MAX.AF.POPS,SIFT,PolyPhen,SIFTprediction,PolyPhenprediction,TRANSCRIPTION.FACTORS,miRNA,DOMAINS,MOTIF.NAME,MOTIF.POS,MOTIF.SCORE.CHANGE,HIGH.INF.POS
2571266,P732912,chr18_50872497_G/A,-,True,HET,SNV,intergenic_variant,-,True,VUS,Common (>=10%),Common (>=5%),uncertain_significance,0.54180,gnomADg_AMI,-,-,-,-,-,-,-,-,-,-,-
2571255,P732912,chr18_50866180_C/T,-,True,HET,SNV,regulatory_region_variant,-,True,VUS,Common (>=10%),Common (>=5%),uncertain_significance,0.54070,gnomADg_AMI,-,-,-,-,-,-,-,-,-,-,-
2571244,P732912,chr18_50861586_G/A,-,True,HET,SNV,regulatory_region_variant,-,True,VUS,Common (>=10%),Common (>=5%),uncertain_significance,0.54060,gnomADg_AMI,-,-,-,-,-,-,-,-,-,-,-
2364932,P732912,chr16_16075709_C/T,ABCC1,True,HET,SNV,intron_variant,-,True,VUS,Common (>=10%),Common (>=5%),uncertain_significance,0.76370,gnomADg_AMI,-,-,-,-,-,-,-,-,-,-,-
1804022,P732912,chr11_17395957_A/G,ABCC8,True,HET,SNV,intron_variant,-,True,VUS,Common (>=10%),Common (>=5%),"uncertain_significance,benign,likely_benign",0.78740,AFR,-,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89329,P732912,chr3_10152061_AAAA/-/AAA,VHL,True,HET,sequence_alteration,3_prime_UTR_variant,-,False,VUS,Common (>=10%),Common (>=5%),uncertain_significance,0.33950,gnomADg_AFR,-,-,NaN,NaN,-,-,-,-,-,-,-
1935186,P732912,chr12_6063036_T/C,VWF,True,HOM,SNV,missense_variant,H/R,True,VUS,Common (>=10%),Common (>=5%),"benign,uncertain_significance,likely_benign",0.74030,SAS,tolerated(1),benign(0.026),tolerated,benign,-,-,"PDB-ENSP_mappings:7kwo.V,PDB-ENSP_mappings:7pm...",-,-,-,-
542466,P732912,chrX_123911478_-/A,XIAP,True,HET,insertion,3_prime_UTR_variant,-,True,VUS,Common (>=10%),Common (>=5%),"uncertain_significance,benign",0.52440,AFR,-,-,NaN,NaN,-,-,-,-,-,-,-
524906,P732912,chr22_40929193_-/T,XPNPEP3,True,HET,insertion,3_prime_UTR_variant,-,False,VUS,Less common (<10%),Rare (<5%),uncertain_significance,0.02912,gnomADg_ASJ,-,-,NaN,NaN,-,-,-,-,-,-,-


In [35]:
checkColumnValue(cad63, 'ClinVar')

,Count,count
0,NaN,3667535
1,BENIGN,33116
2,VUS,346
3,PATHO,17


In [36]:

patho63 = cad63[cad63['ClinVar'] == 'PATHO']
print(patho63.shape)
patho63[keepme].sort_values(['SYMBOL'])

(17, 95)


,IND,Uploaded.variation,SYMBOL,gene.in.both.lines,ZYG,VARIANT.CLASS,Consequence,Amino.acids,var.in.both.lines,ClinVar,MAF,MAF2,CLIN.SIG,MAX.AF,MAX.AF.POPS,SIFT,PolyPhen,SIFTprediction,PolyPhenprediction,TRANSCRIPTION.FACTORS,miRNA,DOMAINS,MOTIF.NAME,MOTIF.POS,MOTIF.SCORE.CHANGE,HIGH.INF.POS
23723,P931263,chr1_18872954_G/T,ALDH4A1,True,HET,SNV,missense_variant,T/N,False,PATHO,Less common (<10%),Rare (<5%),conflicting_interpretations_of_pathogenicity,0.042630,gnomADg_FIN,deleterious(0),probably_damaging(0.97),deleterious,probably_damaging,-,-,"PDB-ENSP_mappings:3v9g.A,PDB-ENSP_mappings:3v9...",-,-,-,-
284152,P931263,chr9_27573524_GCCCCG/-,C9orf72,True,HOM,deletion,upstream_gene_variant,-,False,PATHO,Common (>=10%),Common (>=5%),pathogenic,0.657700,gnomADg_ASJ,-,-,NaN,NaN,-,-,-,-,-,-,-
2408364,P931263,chr14_91297376_G/A,CCDC88C,True,HET,SNV,missense_variant,R/C,False,PATHO,Rare (<1%),Rare (<5%),conflicting_interpretations_of_pathogenicity,0.007261,gnomADg_NFE,deleterious(0),probably_damaging(0.973),deleterious,probably_damaging,-,-,"PANTHER:PTHR18947,PANTHER:PTHR18947:SF31,Coile...",-,-,-,-
2653430,P931263,chr17_34252769_A/G,CCL2,True,HET,SNV,upstream_gene_variant,-,False,PATHO,Common (>=10%),Common (>=5%),"risk_factor,pathogenic",0.555700,gnomADg_EAS,-,-,-,-,-,-,-,-,-,-,-
543951,P931263,chr3_39265671_G/A,CX3CR1,True,HET,SNV,missense_variant,T/M,False,PATHO,Common (>=10%),Common (>=5%),"risk_factor,pathogenic,protective",0.186600,gnomADe_ASJ,deleterious(0.01),probably_damaging(0.978),deleterious,probably_damaging,-,-,"Gene3D:1.20.1070.10,PDB-ENSP_mappings:7xbw.R,P...",-,-,-,-
543952,P931263,chr3_39265765_C/T,CX3CR1,True,HET,SNV,missense_variant,V/I,False,PATHO,Common (>=10%),Common (>=5%),"pathogenic,risk_factor,protective",0.285300,"EUR,gnomADg_MID",tolerated(0.61),benign(0.012),tolerated,benign,-,-,"Gene3D:1.20.1070.10,PDB-ENSP_mappings:7xbw.R,P...",-,-,-,-
484803,P931263,chr2_232486499_A/G,ECEL1,True,HET,SNV,missense_variant,L/P,False,PATHO,Rare (<1%),Rare (<5%),conflicting_interpretations_of_pathogenicity,0.007882,gnomADg_FIN,tolerated(0.05),possibly_damaging(0.472),tolerated,possibly_damaging,-,-,"PANTHER:PTHR11733:SF195,PANTHER:PTHR11733,AFDB...",-,-,-,-
315308,P931263,chr2_62904596_G/A,EHBP1,True,HET,SNV,intron_variant,-,False,PATHO,Common (>=10%),Common (>=5%),pathogenic,0.185900,AMR,-,-,-,-,-,-,-,-,-,-,-
1142271,P931263,chr6_26090951_C/G,HFE,True,HET,SNV,missense_variant,H/D,False,PATHO,Common (>=10%),Common (>=5%),"risk_factor,not_provided,conflicting_interpret...",0.172000,EUR,tolerated(0.09),possibly_damaging(0.704),tolerated,possibly_damaging,-,-,"PDB-ENSP_mappings:1a6z.A,PDB-ENSP_mappings:1a6...",-,-,-,-
2550959,P931263,chr16_27344882_A/G,IL4R,True,HET,SNV,missense_variant,I/V,False,PATHO,Common (>=10%),Common (>=5%),"protective,pathogenic",0.524900,gnomADg_EAS,tolerated(1),benign(0.001),tolerated,benign,-,-,"PDB-ENSP_mappings:1iar.B,Gene3D:2.60.40.10,PDB...",-,-,-,-


In [37]:
# patho63 = cad63[cad63['ClinVar'] == 'PATHO']
# print(patho63.shape)
# patho63[keepme].sort_values(['SYMBOL'])

benign63 = cad63[cad63['ClinVar'] == 'BENIGN']
print(benign63.shape)
benign63[keepme].sort_values(['SYMBOL'])

(33116, 95)


,IND,Uploaded.variation,SYMBOL,gene.in.both.lines,ZYG,VARIANT.CLASS,Consequence,Amino.acids,var.in.both.lines,ClinVar,MAF,MAF2,CLIN.SIG,MAX.AF,MAX.AF.POPS,SIFT,PolyPhen,SIFTprediction,PolyPhenprediction,TRANSCRIPTION.FACTORS,miRNA,DOMAINS,MOTIF.NAME,MOTIF.POS,MOTIF.SCORE.CHANGE,HIGH.INF.POS
257253,P931263,chr2_11741072_A/C,-,True,HET,SNV,non_coding_transcript_exon_variant,-,False,BENIGN,Common (>=10%),Common (>=5%),benign,0.6846,AFR,-,-,-,-,-,-,-,-,-,-,-
1678588,P931263,chr9_69099777_C/T,-,True,HET,SNV,"intron_variant,NMD_transcript_variant",-,False,BENIGN,Common (>=10%),Common (>=5%),benign,0.6136,gnomADg_EAS,-,-,-,-,-,-,-,-,-,-,-
1678587,P931263,chr9_69099647_G/A,-,True,HET,SNV,"intron_variant,NMD_transcript_variant",-,False,BENIGN,Common (>=10%),Common (>=5%),benign,0.6136,gnomADg_EAS,-,-,-,-,-,-,-,-,-,-,-
2222330,P931263,chr13_20366042_T/C,-,True,HOM,SNV,regulatory_region_variant,-,True,BENIGN,Common (>=10%),Common (>=5%),benign,0.7751,gnomADg_FIN,-,-,-,-,-,-,-,-,-,-,-
2222331,P931263,chr13_20366194_C/G,-,True,HOM,SNV,regulatory_region_variant,-,True,BENIGN,Common (>=10%),Common (>=5%),benign,0.7749,gnomADg_FIN,-,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989614,P931263,chr5_61531912_T/G,ZSWIM6,True,HET,SNV,intron_variant,-,False,BENIGN,Common (>=10%),Common (>=5%),benign,0.4593,gnomADg_AMI,-,-,-,-,-,-,-,-,-,-,-
989615,P931263,chr5_61535306_C/T,ZSWIM6,True,HET,SNV,intron_variant,-,False,BENIGN,Common (>=10%),Common (>=5%),benign,0.2719,gnomADg_AMI,-,-,-,-,-,-,-,-,-,-,-
989616,P931263,chr5_61535754_T/G,ZSWIM6,True,HET,SNV,intron_variant,-,True,BENIGN,Common (>=10%),Common (>=5%),benign,0.4594,gnomADg_AMI,-,-,-,-,-,-,-,-,-,-,-
989618,P931263,chr5_61539194_T/C,ZSWIM6,True,HET,SNV,intron_variant,-,True,BENIGN,Common (>=10%),Common (>=5%),benign,0.4632,SAS,-,-,-,-,-,-,-,-,-,-,-


In [38]:
# patho63 = cad63[cad63['ClinVar'] == 'PATHO']
# print(patho63.shape)
# patho63[keepme].sort_values(['SYMBOL'])

# benign63 = cad63[cad63['ClinVar'] == 'BENIGN']
# print(benign63.shape)
# benign63[keepme].sort_values(['SYMBOL'])

vus63 = cad63[cad63['ClinVar'] == 'VUS']
print(vus63.shape)
vus63[keepme].sort_values(['SYMBOL'])

(346, 95)


,IND,Uploaded.variation,SYMBOL,gene.in.both.lines,ZYG,VARIANT.CLASS,Consequence,Amino.acids,var.in.both.lines,ClinVar,MAF,MAF2,CLIN.SIG,MAX.AF,MAX.AF.POPS,SIFT,PolyPhen,SIFTprediction,PolyPhenprediction,TRANSCRIPTION.FACTORS,miRNA,DOMAINS,MOTIF.NAME,MOTIF.POS,MOTIF.SCORE.CHANGE,HIGH.INF.POS
2752162,P931263,chr18_50866180_C/T,-,True,HET,SNV,regulatory_region_variant,-,True,VUS,Common (>=10%),Common (>=5%),uncertain_significance,0.5407,gnomADg_AMI,-,-,-,-,-,-,-,-,-,-,-
2752175,P931263,chr18_50872497_G/A,-,True,HET,SNV,intergenic_variant,-,True,VUS,Common (>=10%),Common (>=5%),uncertain_significance,0.5418,gnomADg_AMI,-,-,-,-,-,-,-,-,-,-,-
2752151,P931263,chr18_50861586_G/A,-,True,HET,SNV,regulatory_region_variant,-,True,VUS,Common (>=10%),Common (>=5%),uncertain_significance,0.5406,gnomADg_AMI,-,-,-,-,-,-,-,-,-,-,-
1259993,P931263,chr6_151630085_A/G,-,True,HET,SNV,regulatory_region_variant,-,False,VUS,Common (>=10%),Common (>=5%),uncertain_significance,0.3125,EAS,-,-,-,-,-,-,-,-,-,-,-
2541083,P931263,chr16_16075709_C/T,ABCC1,True,HET,SNV,intron_variant,-,True,VUS,Common (>=10%),Common (>=5%),uncertain_significance,0.7637,gnomADg_AMI,-,-,-,-,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446768,P931263,chr16_46659710_-/A,VPS35,True,HET,insertion,3_prime_UTR_variant,-,False,VUS,Common (>=10%),Common (>=5%),"uncertain_significance,likely_benign",0.3242,AMR,-,-,NaN,NaN,-,-,-,-,-,-,-
2079362,P931263,chr12_5985618_T/C,VWF,True,HET,SNV,synonymous_variant,T,False,VUS,Common (>=10%),Common (>=5%),"benign,uncertain_significance,likely_benign",0.1969,gnomADe_AFR,-,-,-,-,-,-,"Pfam:PF00093,PIRSF:PIRSF002495,SMART:SM00214,P...",-,-,-,-
2079587,P931263,chr12_6063036_T/C,VWF,True,HOM,SNV,missense_variant,H/R,True,VUS,Common (>=10%),Common (>=5%),"benign,uncertain_significance,likely_benign",0.7403,SAS,tolerated(1),benign(0.026),tolerated,benign,-,-,"PDB-ENSP_mappings:7kwo.V,PDB-ENSP_mappings:7pm...",-,-,-,-
558471,P931263,chrX_123911478_-/A,XIAP,True,HET,insertion,3_prime_UTR_variant,-,True,VUS,Common (>=10%),Common (>=5%),"uncertain_significance,benign",0.5244,AFR,-,-,NaN,NaN,-,-,-,-,-,-,-


---

# [4] label Database - indicates if variant is found in ClinVar and Gnomad, or only ClinVar, or only Gnomad
 
---


In [39]:
# create a new column 'Database' in both dataframes, if MAX.AF is not NA and ClinVar is NA label 'gnomAD', if ClinVar is not NA and MAX.AF is NA label 'ClinVar', if both are not NA label 'gnomAD & ClinVar', else label 'No DB source'
cad12['Database'] = np.select([cad12['ClinVar'].notna() & cad12['MAX.AF'].isna(), 
                                  cad12['MAX.AF'].notna() & cad12['ClinVar'].isna(), 
                                  cad12['MAX.AF'].notna() & cad12['ClinVar'].notna(), 
                                  cad12['MAX.AF'].isna() & cad12['ClinVar'].isna()], 
                                  ['ClinVar', 'gnomAD', 'gnomAD & ClinVar', 'No DB source'], 
                                  default = np.nan)

cad63['Database'] = np.select([cad63['ClinVar'].notna() & cad63['MAX.AF'].isna(), 
                                  cad63['MAX.AF'].notna() & cad63['ClinVar'].isna(), 
                                  cad63['MAX.AF'].notna() & cad63['ClinVar'].notna(), 
                                  cad63['MAX.AF'].isna() & cad63['ClinVar'].isna()], 
                                  ['ClinVar', 'gnomAD', 'gnomAD & ClinVar', 'No DB source'], 
                                  default = np.nan)


In [40]:
checkColumnValue(cad12, 'Database')

,Count,count
0,gnomAD,3428264
1,No DB source,35778
2,gnomAD & ClinVar,32158
3,ClinVar,1


In [41]:
checkColumnValue(cad63, 'Database')

,Count,count
0,gnomAD,3641404
1,gnomAD & ClinVar,33479
2,No DB source,26131


In [42]:
crosstabit(cad12, 'MAF', 'Database', True)

Database,ClinVar,No DB source,gnomAD,gnomAD & ClinVar,Total
MAF,,,,,
Common (>=10%),0,0,3221872,31160,3253032
Less common (<10%),0,0,152004,856,152860
Rare (<1%),0,0,54388,142,54530
nan,1,35778,0,0,35779
Total,1,35778,3428264,32158,3496201


In [43]:
crosstabit(cad63, 'MAF', 'Database', True)

Database,No DB source,gnomAD,gnomAD & ClinVar,Total
MAF,,,,
Common (>=10%),0,3430130,32339,3462469
Less common (<10%),0,156632,992,157624
Rare (<1%),0,54642,148,54790
nan,26131,0,0,26131
Total,26131,3641404,33479,3701014


In [44]:
crosstabit(cad12, 'ClinVar', 'Database', True)


Database,ClinVar,No DB source,gnomAD,gnomAD & ClinVar,Total
ClinVar,,,,,
BENIGN,1,0,0,31834,31835
NA,0,35778,3428264,0,3464042
PATHO,0,0,0,15,15
VUS,0,0,0,309,309
Total,1,35778,3428264,32158,3496201


In [45]:
crosstabit(cad63, 'ClinVar', 'Database', True)

Database,No DB source,gnomAD,gnomAD & ClinVar,Total
ClinVar,,,,
BENIGN,0,0,33116,33116
NA,26131,3641404,0,3667535
PATHO,0,0,17,17
VUS,0,0,346,346
Total,26131,3641404,33479,3701014


---

# SAVE individual dfs

---

In [46]:

print(cad12.shape)
print(cad63.shape)
print(cad12.columns)

(3496201, 96)
(3701014, 96)
Index(['Uploaded.variation', 'Location', 'Allele', 'Gene', 'Feature', 'Feature.type', 'Consequence', 'cDNA.position', 'CDS.position', 'Protein.position', 'Amino.acids', 'Codons', 'Existing.variation', 'IND', 'ZYG', 'REF.ALLELE', 'IMPACT', 'DISTANCE', 'STRAND', 'FLAGS', 'VARIANT.CLASS', 'SYMBOL', 'SYMBOL.SOURCE', 'HGNC.ID', 'BIOTYPE', 'CANONICAL', 'MANE.SELECT', 'MANE.PLUS.CLINICAL', 'TSL', 'APPRIS', 'CCDS', 'ENSP', 'SWISSPROT', 'TREMBL', 'UNIPARC', 'UNIPROT.ISOFORM', 'REFSEQ.MATCH', 'SOURCE', 'REFSEQ.OFFSET', 'GIVEN.REF', 'USED.REF', 'BAM.EDIT', 'GENE.PHENO', 'SIFT', 'PolyPhen', 'EXON', 'INTRON', 'DOMAINS', 'miRNA', 'HGVSc', 'HGVSp', 'HGVS.OFFSET', 'gnomADg.AF', 'gnomADg.AFR.AF', 'gnomADg.AMI.AF', 'gnomADg.AMR.AF', 'gnomADg.ASJ.AF', 'gnomADg.EAS.AF', 'gnomADg.FIN.AF', 'gnomADg.MID.AF', 'gnomADg.NFE.AF', 'gnomADg.OTH.AF', 'gnomADg.SAS.AF', 'MAX.AF', 'MAX.AF.POPS', 'CLIN.SIG', 'SOMATIC', 'PHENO', 'PUBMED', 'MOTIF.NAME', 'MOTIF.POS', 'HIGH.INF.POS', 'MOTIF.SCOR

In [47]:
cad12.describe()

,DISTANCE,gnomADg.AF,gnomADg.AFR.AF,gnomADg.AMI.AF,gnomADg.AMR.AF,gnomADg.ASJ.AF,gnomADg.EAS.AF,gnomADg.FIN.AF,gnomADg.MID.AF,gnomADg.NFE.AF,gnomADg.OTH.AF,gnomADg.SAS.AF,MAX.AF,chrCoordinate,EXONnumber,INTRONnumber,SIFTscore,PolyPhenscore
count,436902.000000,3.460261e+06,3.460259e+06,3.460165e+06,3.460258e+06,3.460242e+06,3.460250e+06,3.460254e+06,3.460100e+06,3.460261e+06,3.460238e+06,3.460252e+06,3.460422e+06,3.496201e+06,85969.000000,1.887824e+06,8742.000000,8444.000000
mean,2488.211519,4.589795e-01,4.377915e-01,4.657630e-01,4.650005e-01,4.667297e-01,4.721054e-01,4.663075e-01,4.652989e-01,4.663965e-01,4.609162e-01,4.762419e-01,6.084945e-01,7.623309e+07,6.972944,5.257583e+00,0.460385,0.152510
std,1451.121397,2.896428e-01,3.017765e-01,3.143898e-01,2.971199e-01,3.031336e-01,3.280300e-01,3.066453e-01,2.999533e-01,3.037308e-01,2.917207e-01,2.960810e-01,2.951332e-01,5.659873e+07,9.018434,7.827981e+00,0.400754,0.301893
min,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.018500e+04,1.000000,1.000000e+00,0.000000,0.000000
25%,1226.000000,2.150000e-01,1.696000e-01,1.930000e-01,2.125000e-01,2.075000e-01,1.786000e-01,2.022000e-01,2.092000e-01,2.064000e-01,2.148000e-01,2.275000e-01,3.797000e-01,2.953120e+07,2.000000,1.000000e+00,0.070000,0.000000
50%,2483.000000,4.385000e-01,4.056000e-01,4.458000e-01,4.459000e-01,4.472000e-01,4.533000e-01,4.461000e-01,4.459000e-01,4.464000e-01,4.407000e-01,4.572000e-01,6.359000e-01,6.551204e+07,4.000000,2.000000e+00,0.340000,0.001000
75%,3746.000000,6.884000e-01,6.928000e-01,7.248000e-01,7.022000e-01,7.100000e-01,7.547000e-01,7.154000e-01,7.057000e-01,7.100000e-01,6.921000e-01,7.127000e-01,8.767000e-01,1.131298e+08,9.000000,6.000000e+00,1.000000,0.084000
max,5000.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.489456e+08,358.000000,3.470000e+02,1.000000,1.000000


In [48]:
cad63.describe()

,DISTANCE,gnomADg.AF,gnomADg.AFR.AF,gnomADg.AMI.AF,gnomADg.AMR.AF,gnomADg.ASJ.AF,gnomADg.EAS.AF,gnomADg.FIN.AF,gnomADg.MID.AF,gnomADg.NFE.AF,gnomADg.OTH.AF,gnomADg.SAS.AF,MAX.AF,chrCoordinate,EXONnumber,INTRONnumber,SIFTscore,PolyPhenscore
count,444320.000000,3.674811e+06,3.674809e+06,3.674742e+06,3.674810e+06,3.674806e+06,3.674802e+06,3.674804e+06,3.674681e+06,3.674811e+06,3.674795e+06,3.674798e+06,3.674883e+06,3.701014e+06,87596.000000,2.002513e+06,8629.000000,8311.000000
mean,2488.533665,4.718748e-01,4.426008e-01,4.853570e-01,4.760430e-01,4.827356e-01,4.682483e-01,4.841243e-01,4.789125e-01,4.857458e-01,4.744601e-01,4.761085e-01,6.144999e-01,7.697594e+07,7.139847,5.279100e+00,0.462458,0.148457
std,1450.436258,2.861300e-01,3.019205e-01,3.081501e-01,2.947012e-01,2.982019e-01,3.327319e-01,3.008077e-01,2.962162e-01,2.967645e-01,2.878847e-01,3.004508e-01,2.933959e-01,5.649273e+07,9.421144,7.897302e+00,0.402552,0.296801
min,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.028600e+04,1.000000,1.000000e+00,0.000000,0.000000
25%,1228.000000,2.346000e-01,1.747000e-01,2.236000e-01,2.289000e-01,2.330000e-01,1.654000e-01,2.302000e-01,2.310000e-01,2.377000e-01,2.355000e-01,2.235000e-01,3.898000e-01,3.043392e+07,2.000000,1.000000e+00,0.070000,0.000000
50%,2483.000000,4.546000e-01,4.114000e-01,4.704000e-01,4.589000e-01,4.667000e-01,4.495000e-01,4.679000e-01,4.620000e-01,4.699000e-01,4.569000e-01,4.585000e-01,6.432000e-01,6.700125e+07,4.000000,3.000000e+00,0.350000,0.001000
75%,3746.000000,6.991000e-01,6.991000e-01,7.390000e-01,7.122000e-01,7.220000e-01,7.574000e-01,7.283000e-01,7.179000e-01,7.241000e-01,7.032000e-01,7.185000e-01,8.813000e-01,1.130781e+08,9.000000,6.000000e+00,1.000000,0.078000
max,5000.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.489456e+08,358.000000,3.540000e+02,1.000000,1.000000


In [49]:
describeMe(cad12)
describeMe(cad63)

(3496201, 96)
Uploaded.variation           object
Location                     object
Allele                       object
Gene                         object
Feature                      object
Feature.type                 object
Consequence                  object
cDNA.position                object
CDS.position                 object
Protein.position             object
Amino.acids                  object
Codons                       object
Existing.variation           object
IND                          object
ZYG                          object
REF.ALLELE                   object
IMPACT                       object
DISTANCE                    float64
STRAND                       object
FLAGS                        object
VARIANT.CLASS                object
SYMBOL                       object
SYMBOL.SOURCE                object
HGNC.ID                      object
BIOTYPE                      object
CANONICAL                    object
MANE.SELECT                  object
MANE.PLUS.CLIN

<function all_funx.describeMe(df)>

In [50]:
cad12.to_csv('4-cad12-snv-indel-3496201.csv', index=False)
cad63.to_csv('4-cad63-snv-indel-3701014.csv', index=False)

# TODO 
# 1. (done) note the clinvar col with labels is not star reviewed, check the clin.sig column to confirm high confidence pathogenicity
# 2. (done) added Database column here instead of in R to keep all label functions together in python

---

# SAVE both individuals df

---

In [51]:
simple12 = cad12[keepme].copy()
simple63 = cad63[keepme].copy()


both = pd.concat([simple12, simple63], axis=0)
print(both.shape)# (7197215, 32)

both.to_csv('4-keepme-combined-cad12-cad63-7197215.csv', index=False)

(7197215, 26)
